In [72]:
import numpy as np
import pandas as pd
# 绘图
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

import warnings

warnings.filterwarnings('ignore')

train_df = pd.read_csv('cy_kaggle/data/train.csv')
test_df = pd.read_csv('cy_kaggle/data/test.csv')
combine_df = pd.concat([train_df, test_df])

train_df.groupby(train_df.Name.apply(lambda x: len(x)))['Survived'].mean().plot()
combine_df['Name_len'] = combine_df['Name'].apply(lambda x: len(x))
combine_df['Name_len'] = pd.qcut(combine_df['Name_len'], 5)

combine_df.groupby(combine_df['Name']
                   .apply(lambda x: x.split(', ')[1])
                   .apply(lambda x: x.split('.')[0]))['Survived'].mean().plot()

combine_df['Title'] = combine_df['Name'].apply(lambda x: x.split(', ')[1]).apply(lambda x: x.split('.')[0])
combine_df['Title'] = combine_df['Title'].replace(
    ['Don', 'Dona', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col', 'Sir', 'Dr'], 'Mr')
combine_df['Title'] = combine_df['Title'].replace(['Mlle', 'Ms'], 'Miss')
combine_df['Title'] = combine_df['Title'].replace(['the Countess', 'Mme', 'Lady', 'Dr'], 'Mrs')
df = pd.get_dummies(combine_df['Title'], prefix='Title')
combine_df = pd.concat([combine_df, df], axis=1)
combine_df['Fname'] = combine_df['Name'].apply(lambda x: x.split(',')[0])
combine_df['Familysize'] = combine_df['SibSp'] + combine_df['Parch']
dead_female_Fname = list(set(combine_df[(combine_df.Sex == 'female') & (combine_df.Age >= 12)
                                        & (combine_df.Survived == 0) & (combine_df.Familysize > 1)]['Fname'].values))
survive_male_Fname = list(set(combine_df[(combine_df.Sex == 'male') & (combine_df.Age >= 12)
                                         & (combine_df.Survived == 1) & (combine_df.Familysize > 1)]['Fname'].values))
print(dead_female_Fname)
combine_df['Dead_female_family'] = np.where(combine_df['Fname'].isin(dead_female_Fname), 1, 0)
combine_df['Survive_male_family'] = np.where(combine_df['Fname'].isin(survive_male_Fname), 1, 0)
combine_df = combine_df.drop(['Name', 'Fname'], axis=1)

group = combine_df.groupby(['Title', 'Pclass'])['Age']
combine_df['Age'] = group.transform(lambda x: x.fillna(x.median()))
combine_df = combine_df.drop('Title', axis=1)
combine_df['IsChild'] = np.where(combine_df['Age'] <= 12, 1, 0)
combine_df['Age'] = pd.cut(combine_df['Age'], 5)
combine_df = combine_df.drop('Age', axis=1)
# combine_df['Familysize'] = np.where(combine_df['Familysize'] == 0, 'solo',
#                                     np.where(combine_df['Familysize'] <= 3, 'normal', 'big'))
# df = pd.get_dummies(combine_df['Familysize'], prefix='Familysize')
# combine_df = pd.concat([combine_df, df], axis=1).drop(['SibSp', 'Parch', 'Familysize'], axis=1)

combine_df['Low_Survival_Familysize'] = np.where(combine_df['Familysize'].isin([7, 8, 9, 10]), 1, 0)
combine_df['Middle_Survival_Familysize'] = np.where(combine_df['Familysize'].isin([0, 4, 5, 6]), 1, 0)
combine_df['High_Survival_Familysize'] = np.where(combine_df['Familysize'].isin([1, 2, 3]), 1, 0)
combine_df = combine_df.drop(['SibSp', 'Parch', 'Familysize'], axis=1)

combine_df['Ticket_Lett'] = combine_df['Ticket'].apply(lambda x: str(x)[0])
combine_df['Ticket_Lett'] = combine_df['Ticket_Lett'].apply(lambda x: str(x))

combine_df['High_Survival_Ticket'] = np.where(combine_df['Ticket_Lett'].isin(['1', '2', 'P']), 1, 0)
combine_df['Low_Survival_Ticket'] = np.where(combine_df['Ticket_Lett'].isin(['A', 'W', '3', '7']), 1, 0)
combine_df = combine_df.drop(['Ticket', 'Ticket_Lett'], axis=1)
combine_df.Embarked = combine_df.Embarked.fillna('S')
df = pd.get_dummies(combine_df['Embarked'], prefix='Embarked')
combine_df = pd.concat([combine_df, df], axis=1).drop('Embarked', axis=1)

combine_df.Cabin = combine_df.Cabin.fillna('None')
combine_df['Cabin_type'] = combine_df.Cabin.apply(lambda x: x[0])
# df = pd.get_dummies(combine_df['Cabin_type'], prefix='Cabin')
# combine_df = pd.concat([combine_df, df], axis=1).drop('Cabin_type', axis=1)

combine_df['High_Survival_Cabin'] = np.where(combine_df['Cabin_type'].isin(['B', 'D', 'E']), 1, 0)
combine_df['Middle_Survival_Cabin'] = np.where(combine_df['Cabin_type'].isin(['A', 'C', 'F', 'G']), 1, 0)
combine_df['Low_Survival_Cabin'] = np.where(combine_df['Cabin_type'].isin(['N', 'T']), 1, 0)

#combine_df['Cabin_isNull'] = np.where(combine_df['Cabin'].isnull(), 0, 1)
combine_df = combine_df.drop('Cabin', axis=1)
df = pd.get_dummies(combine_df['Pclass'], prefix='Pclass')
combine_df = pd.concat([combine_df, df], axis=1).drop('Pclass', axis=1)
df = pd.get_dummies(combine_df['Sex'], prefix='Sex')
combine_df = pd.concat([combine_df, df], axis=1).drop('Sex', axis=1)
combine_df['Fare'] = pd.qcut(combine_df.Fare, 3)
df = pd.get_dummies(combine_df.Fare, prefix='Fare').drop('Fare_(-0.001, 8.662]', axis=1)
#df = pd.get_dummies(combine_df.Fare, prefix='Fare')
combine_df = pd.concat([combine_df, df], axis=1).drop('Fare', axis=1)

features = combine_df.drop(["PassengerId", "Survived"], axis=1).columns
combine_df.info()
le = LabelEncoder()
for feature in features:
    le = le.fit(combine_df[feature])
    combine_df[feature] = le.transform(combine_df[feature])
X_all = combine_df.iloc[:891, :].drop(["PassengerId", "Survived"], axis=1)
Y_all = combine_df.iloc[:891, :]["Survived"]
X_test = combine_df.iloc[891:, :].drop(["PassengerId", "Survived"], axis=1)


LR 0.8484856996935648
SVC 0.8372244921121327
KNN 0.8383985926682556
decision_tree 0.831682555896039
random_forest 0.8272749971626376
GBDT 0.848473215299058


In [78]:
X_all.rename(
    columns={'Fare_(-0.001, 8.662]': 'Fare_0', 'Fare_(8.662, 26.0]': 'Fare_1', 'Fare_(26.0, 512.329]': 'Fare_2'},
    inplace=True)

X_test.rename(
    columns={'Fare_(-0.001, 8.662]': 'Fare_0', 'Fare_(8.662, 26.0]': 'Fare_1', 'Fare_(26.0, 512.329]': 'Fare_2'},
    inplace=True)

lr = LogisticRegression()
svc = SVC()
knn = KNeighborsClassifier(n_neighbors=3)
dt = DecisionTreeClassifier()
rf = RandomForestClassifier(n_estimators=300, min_samples_leaf=4, class_weight={0: 0.745, 1: 0.255})
gbdt = GradientBoostingClassifier(n_estimators=500, learning_rate=0.03, max_depth=3)
xgbGBDT = XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)
clfs = [lr, svc, knn, dt, rf, gbdt, xgbGBDT]

kfold = 10
cv_results = []
for classifier in clfs:
    cv_results.append(cross_val_score(classifier, X_all, y=Y_all, scoring="accuracy", cv=kfold, n_jobs=4))

cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

ag = ["LR", "SVC", 'KNN', 'decision_tree', "random_forest", "GBDT", "xgbGBDT"]
cv_res = pd.DataFrame({"CrossValMeans": cv_means, "CrossValerrors": cv_std,
                       "Algorithm": ag})

g = sns.barplot("CrossValMeans", "Algorithm", data=cv_res, palette="Set3", orient="h", **{'xerr': cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")


class Ensemble(object):
    def __init__(self, estimators):
        self.estimatos_names = []
        self.estimatos = []
        for i in estimators:
            self.estimatos_names.append(i[0])
            self.estimatos.append(i[1])
        self.clf = LogisticRegression()
    def fit(self, train_X, train_y):
        for i in self.estimatos:
            i.fit(train_X, train_y)
        x = np.array([i.predict(train_X) for i in self.estimatos]).T
        y = train_y
        self.clf.fit(x, y)
    def predict(self, x):
        x = np.array([i.predict(x) for i in self.estimatos]).T
        # print(x)
        return self.clf.predict(x)
    def score(self, x, y):
        s = precision_score(y, self.predict(x))
        return s


LR 0.8484856996935648
SVC 0.8372244921121327
KNN 0.8383985926682556
decision_tree 0.831682555896039
random_forest 0.8272749971626376
GBDT 0.848473215299058


In [79]:
for i in range(6):
    print(ag[i], cv_means[i])

LR 0.8484856996935648
SVC 0.8372244921121327
KNN 0.8383985926682556
decision_tree 0.831682555896039
random_forest 0.8272749971626376
GBDT 0.848473215299058


In [81]:
bag = Ensemble([('xgb', xgbGBDT), ('lr', lr), ('rf', rf), ('svc', svc), ('gbdt', gbdt)])
score = 0
for i in range(0, 10):
    num_test = 0.20
    X_train, X_cv, Y_train, Y_cv = train_test_split(X_all, Y_all, test_size=num_test)
    bag.fit(X_train, Y_train)
    #Y_test = bag.predict(X_test)
    acc_xgb = round(bag.score(X_cv, Y_cv) * 100, 2)
    score += acc_xgb
print(score / 10)  #0.8786

bag.fit(X_all,Y_all)
y_predict = bag.predict(X_test)
with open('cy_kaggle/data/new_svc_result.csv', 'w') as result_file:
    result_file.write("PassengerId,Survived" + "\n")
    for pid, flag in zip(test_df.values[:, 0], y_predict):
        result_file.write(str(pid) + "," + str(int(flag)) + "\n")


81.832
